# Parallel Computing
Julia has multiple ways of doing parallel computations. There's experimental multi-threading support and support for distributed computing. We'll touch upon the basics here to give you an idea what's possible.

## Threading

Threading is built-in nowadays, and we'll ignore the task part here, but go straight for speeding up computations. You can check whether this notebook actually already supports multiple threads:

In [ ]:
Threads.nthreads()

Each thread has its own id, and we can use these. Let's do it in parallel as well.

In [ ]:
a = zeros(Threads.nthreads()*2)
Threads.@threads for i = 1:length(a)
   a[i] = Threads.threadid()
end
a

However, threads are not simple, because you introduce so called race conditions. Each thread on its will do its thing, without synchronizing with other threads. They can all modify the same value, or read values out of order, leading to unpredictable results.

In [ ]:
total = 0
@Threads.threads for i in 1:1000
  global total += 1
end
total

You can prevent this with setting the sum to be an `Atomic` entity, it should only be accessed by one thread at a time. Another way would be synchronizing (using locks), but that introduces more overhead.

In [ ]:
total = Threads.Atomic{Int}(0)
@Threads.threads for i in 1:10000
    Threads.atomic_add!(total, 1)
end
total

## Distributed
Instead of running threads, you can also run multiple Julia processes and let them communicate (or combine them).
Threading knows about your local memory, but the next process doesn't.

https://docs.julialang.org/en/v1/manual/parallel-computing/#Multi-Core-or-Distributed-Processing-1


Let's add two new worker processes, which can be used for computations.

In [ ]:
using Distributed

In [ ]:
addprocs(2)

We can use the `@distributed` macro to distribute this for loop over all worker processes. Workers make copies of the variables used in this loop. So if we want to write to the same Array on the master process, we need to use the package `SharedArrays`.

In [ ]:
using SharedArrays

a = SharedArray(zeros(10))
@info a  # empty array

@distributed for i = 1:10
    a[i] = i
end

In [ ]:
a

For longer running tasks, we can use `pmap`. It takes a function and an iterable. To use functions outside, we use @everywhere to copy these functions to all worker processes.

In [ ]:
addprocs(100)  # don't repeat this cell too much!

@everywhere function slowtask(_)
    sleep(5)
    getpid()
end

A = rand(100)

@time pmap(slowtask, A)

In [ ]:
rmprocs(workers())